# Installing Libraries

In [ ]:
!pip install PyDrive
!pip install httplib2==0.15.0
!pip install google-api-python-client==1.7.11

# Importing Libraries

In [1]:
import os
import tqdm.notebook as tq
from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth
from pydrive.files import GoogleDriveFileList
import socket
import time

# Defining Variables

In [2]:
src_folder_name =r"C:\Users\DELL\Desktop\check"          #Path to Folder/File to be Uploaded
dst_folder_name ='tqdm_upload'                            #Destination Folder in Drive
parent_folder_id='1l_Q4OStP4F9QJehRAukxWWaoV3BvQnE3'      ##Parent folder ID on Gdrive  , a new folder {dst_folder_name} will be created inside Parent folder

cpt = sum([len(files) for r, d, files in os.walk(src_folder_name)])   #Counting Number of files inside Folder and sub folders , Used for Progress Bar
print(f'Total files in folder = {cpt}')


Total files in folder = 8


# Authenticating

In [3]:
gauth = GoogleAuth()
  
# Creates local webserver and auto
# handles authentication.
gauth.LocalWebserverAuth()       
drive = GoogleDrive(gauth)


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=429743599460-odkn6m66mf952idpqi1okut6tsdnilfh.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


# Defining Helper Functions

In [4]:
def get_folder_id(drive, parent_folder_id, folder_name):
    """ 
    Check if destination folder exists and return it's ID
    :param drive: An instance of GoogleAuth
    :param parent_folder_id: the id of the parent of the folder we are uploading the files to
    :param folder_name: the name of the folder in the drive 
    """

    # Auto-iterate through all files in the parent folder.
    file_list = GoogleDriveFileList()
    

    try:
        file_list = drive.ListFile(
            {'q': "'{0}' in parents and trashed=false".format(parent_folder_id)}
                                  ).GetList()

    # Exit if the parent folder doesn't exist
    except googleapiclient.errors.HttpError as err:
        # Parse error message
        message = ast.literal_eval(err.content)['error']['message']
        if message == 'File not found: ':
            print(message + folder_name)
            exit(1)
        # Exit with stacktrace in case of other error
        else:
            raise

    # Find the the destination folder in the parent folder's files
    for file in file_list:
        if file['title'] == folder_name:
#             print('title: %s, id: %s' % (file['title'], file['id']))
            return file['id']



def create_folder(drive, folder_name, parent_folder_id):
    """ 
    Create folder on Google Drive
    :param drive: An instance of GoogleAuth
    :param folder_id: the id of the folder we are uploading the files to
    :param parent_folder_id: the id of the parent of the folder we are uploading the files to
    """
    folder_metadata = {
        'title': folder_name,
        # Define the file type as folder
        'mimeType': 'application/vnd.google-apps.folder',
        # ID of the parent folder        
        'parents': [{"kind": "drive#fileLink", "id": parent_folder_id}]
    }

    folder = drive.CreateFile(folder_metadata)
    folder.Upload()

    # Return folder informations
#     print('title: %s, id: %s' % (folder['title'], folder['id']))
    return folder['id']





def upload_files_in_folder(drive, folder_id, src_folder_name,bar):

    
    """ 
    Recursively upload files from a folder and its subfolders to Google Drive 
    :param drive: An instance of GoogleAuth
    :param folder_id: the id of the folder we are uploading the files to
    :param src_folder_name: the path to the source folder to upload
    """
#     print("\n Folder:", src_folder_name)
     
    for object_name in os.listdir(src_folder_name):

        filepath = src_folder_name+'/'+object_name

        # Check the file's size
        statinfo = os.stat(filepath)

        f = drive.CreateFile(
            {"parents": [{"kind": "drive#fileLink", "id": folder_id}],
            'title': object_name}
            )
        if os.path.isdir(filepath):

            child_folder_id = get_folder_id(drive, folder_id, object_name)

            # Create the folder if it doesn't exists
            if not child_folder_id:
                child_folder_id = create_folder(drive, object_name, folder_id)
            else:
                print('folder {0} already exists'.format(object_name))


            upload_files_in_folder(drive, child_folder_id, filepath,bar)
        else:
            size=os.path.getsize(filepath)
            size_KB=round((size/1000),3)
            print(f'Uploading file: {object_name} , size = {size_KB}KB ')

            f.SetContentFile(filepath)
            
            if (size>0) :
                try:
                    f.Upload()
                    bar.update(1)

                except:
                    print(f' Ran into an issue while uplaoding {object_name} ,  trying again!!!')

                    time.sleep(10)
                    f.Upload()
                    bar.update(1)

            else:
                print(f'ignoring {object_name} because file is empty ')
                bar.update(1)



# Uploading Folders and Files

In [5]:
folder_id = get_folder_id(drive, parent_folder_id, dst_folder_name)
    
# Create the folder if it doesn't exists
if not folder_id:
    folder_id = create_folder(drive, dst_folder_name, parent_folder_id)
else:
    print('folder {0} already exists'.format(dst_folder_name))
    
bar = tq.tqdm(total=cpt)   #Handles Progress bar  
upload_files_in_folder(drive, folder_id, src_folder_name,bar=bar)     #Upload the files   

folder tqdm_upload already exists


  0%|          | 0/8 [00:00<?, ?it/s]

folder dsd already exists
Uploading file: dsd.txt , size = 0.009KB 
Uploading file: empty.txt , size = 0.0KB 
ignoring empty.txt because file is empty 
folder t already exists
Uploading file: empty_1.txt , size = 0.0KB 
ignoring empty_1.txt because file is empty 
Uploading file: full.txt , size = 0.005KB 
folder fdas already exists
folder dsdasdsa already exists
Uploading file: e.txt , size = 0.0KB 
ignoring e.txt because file is empty 
Uploading file: f.txt , size = 0.003KB 
Uploading file: emptyy.txt , size = 0.0KB 
ignoring emptyy.txt because file is empty 
Uploading file: full_1.txt , size = 0.005KB 
